In [24]:
import os
import chromadb
from google import genai
from huggingface_hub import InferenceClient

CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = int(os.getenv("CHROMA_PORT"))

HF_MODEL = "sentence-transformers/all-mpnet-base-v2"
HF_TOKEN = os.environ["HF_TOKEN"]

chroma_client = chromadb.HttpClient(host=CHROMA_HOST, port=CHROMA_PORT)
collection = chroma_client.get_or_create_collection("superheroes")

gemini_client = genai.Client()

hf_client = InferenceClient(provider="auto", api_key=HF_TOKEN)

In [25]:
def embed_text(query: str):
    return hf_client.feature_extraction([query], model=HF_MODEL)[0]

In [26]:
def ask_superheroes(question: str, top_k: int = 50) -> str:
    query_vector = embed_text(question)

    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )
    docs = results["documents"][0]
    context = "\n\n".join(docs)

    prompt = (
        "You are Batman's AI. Use the context below to answer the question. "
        "If a name in the question is slightly misspelled or formatted differently, "
        "try to find the closest matching superhero using their name or aliases. "
        "Improvise if needed."
        "If you cannot confidently answer, say 'I don't know'.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\nAnswer:"
    )


    response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [31]:
# 1. Make sure these exist in your script:
# - `collection` → Chroma collection with superhero docs + embeddings
# - `embed_text(text: str)` → returns HF embedding vector
# - `gemini_client` → from google.genai.Client()

# 2. Ask a question
question = "with the information you have, develop a plan to defeat superman"
answer = ask_superheroes(question)

print("Q:", question)
print("A:", answer)

Q: with the information you have, develop a plan to defeat superman
A: Analyzing the data on Superman, his allies, and his adversaries, the optimal strategy for neutralization involves a multi-pronged approach exploiting his known vulnerabilities and leveraging strategic assets.

**Objective:** Incapacitate Superman with minimal collateral damage and maximum efficiency.

**Phase 1: Preparation and Intelligence Gathering**

1.  **Vulnerability Assessment:** Superman's "Race: Kryptonian" is the primary weakness. This implies susceptibility to Green Kryptonite, red solar radiation, and certain forms of magic.
2.  **Resource Allocation:**
    *   **Kryptonite Acquisition/Synthesis:** Primary focus. Lex Luthor possesses significant expertise and resources in this area. Metallo, powered by Kryptonite, serves as a proof of concept for its application.
    *   **Red Sun Emitter Development:** Develop portable, high-intensity red solar radiation projectors.
    *   **Containment Unit Design:** 